In [ ]:
class Solution:

    def findSmallestRegion(self, regions: List[List[str]], p: str, q: str) -> str:
        # 最近公共祖先
        fas = {}
        for x, *ys in regions:
            for y in ys:
                fas[y] = x
        r1fas = {p}
        while p in fas:
            p = fas[p]
            r1fas.add(p)
        while q not in r1fas:
            q = fas[q]
        return q

In [ ]:
class Solution:

    def findSmallestRegion(self, regions: List[List[str]], region1: str, region2: str) -> str:
        # 最近公共祖先
        ins = defaultdict(int)
        adj = defaultdict(list)
        for x, *ys in regions:
            adj[x] += ys
            for y in ys:
                ins[y] += 1
        root = next(x for x in adj if ins[x] == 0)

        def lca(root, p, q):
            if root in {p, q, None}:
                return root
            sub1 = sub2 = None
            for y in adj[root]:  # 是否在分叉树枝找到 还是在一个树枝里找到
                sub = lca(y, p, q)
                if sub:
                    if sub1: sub2 = sub
                    else: sub1 = sub
            if sub1 and sub2:
                return root
            return sub1 if sub1 else sub2

        return lca(root, region1, region2)

In [ ]:
class Solution:

    def findSmallestRegion(self, regions: List[List[str]], p: str, q: str) -> str:
        # 最近公共祖先 tarjan算法
        roots = {}

        def find(x):
            if roots[x] != x:
                roots[x] = find(roots[x])
            return roots[x]

        ins = defaultdict(int)
        adj = defaultdict(list)
        for x, *ys in regions:
            adj[x] += ys
            for y in ys:
                ins[y] += 1
        root = next(x for x in adj if ins[x] == 0)
        query = {p: q, q: p}
        colors = defaultdict(int)
        ans = None

        def tarjan(x):
            # 入x
            roots[x] = x  # 动态开点
            colors[x] = 1  # 表示进入x
            for y in adj[x]:
                # if colors[y]: continue
                if tarjan(y): return True
                # 离y 即 y -> x
                roots[y] = x  # y -> x
            # 离x
            if x in query and colors[query[x]] == 2:  # x是查询点 且 目标点已经访问过
                nonlocal ans
                ans = find(query[x])
                return True
            colors[x] = 2  # 表示x已经访问完毕

        tarjan(root)
        return ans

In [ ]:
class Solution:

    def findSmallestRegion(self, regions: List[List[str]], p: str, q: str) -> str:
        # 最近公共祖先 倍增算法
        ins = defaultdict(int)
        adj = defaultdict(list)
        pa = defaultdict(list)  # 关键：x节点2**i个祖先
        for x, *ys in regions:
            adj[x] += ys
            for y in ys:
                pa[y].append(x)  # 第一个祖先（父亲）
                ins[y] += 1
        root = next(x for x in adj if ins[x] == 0)
        pa[root].append('')

        j = 0
        allneg = False  # 全无祖先 即全''
        while not allneg:
            j += 1  # 当前操作的 2**j 的j列
            allneg = True
            for x in pa:
                t = pa[fa][j - 1] if (fa := pa[x][j - 1]) != '' else ''
                pa[x].append(t)
                if t != '': allneg = False

        j += 1  # 列宽
        dep = defaultdict(int)  # 深度

        def dfs(x, fa):
            dep[x] = dep[fa] + 1
            for y in adj[x]:
                dfs(y, x)

        dfs(root, '')

        def getka(x, k):  # x的第k个祖先
            for i in range(k.bit_length()):
                if i > j - 1: return ''
                if k >> i & 1:
                    x = pa[x][i]
                if x == '': return ''
            return x

        if dep[p] < dep[q]:  # 保证p在q的下面 深度更深
            p, q = q, p
        d = dep[p] - dep[q]
        p = getka(p, d)  # 统一深度
        if p == q: return p
        for i in range(j - 1, -1, -1):
            if pa[p][i] != pa[q][i]:
                p = pa[p][i]
                q = pa[q][i]
        return pa[p][0]